In [1]:
# ------------------------------------------------------------
# Configuración Inicial del Entorno
# ------------------------------------------------------------
import sys
import os
import pprint
import torch
import pandas as pd

# Ruta base del proyecto (sube un nivel desde el directorio actual)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(BASE_DIR, "src")

# Verificar si `src` está en sys.path, si no, agregarlo
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"'src' agregado a sys.path: {SRC_PATH}")

# Verificar que la carpeta `loaders` exista dentro de `src`
loaders_path = os.path.join(SRC_PATH, "loaders")
if not os.path.exists(loaders_path):
    raise FileNotFoundError(f"No se encontró la carpeta 'loaders' en {SRC_PATH}. Verifica la estructura del proyecto.")

'src' agregado a sys.path: g:\Mi unidad\EnorChile\GIT\practicos-rag\src


In [2]:
# ------------------------------------------------------------
# Importar módulos necesarios
# ------------------------------------------------------------
try:
    from loaders.pdf_processor import process_pdf_directly
    from loaders.table_review import process_and_review_tables
    from loaders.pdf_inspector import inspect_pdf_content
    print("Módulos importados correctamente.")
except ModuleNotFoundError as e:
    print(f"Error al importar módulos: {e}")
    print("--- Rutas actuales en sys.path ---")
    for path in sys.path:
        print(path)
    raise

Módulos importados correctamente.


In [3]:
import importlib.util

# Cargar el módulo `reranking` manualmente
reranking_path = os.path.join(SRC_PATH, "retrievers", "reranking.py")
spec = importlib.util.spec_from_file_location("retrievers.reranking", reranking_path)
reranking = importlib.util.module_from_spec(spec)
spec.loader.exec_module(reranking)

# Cargar el módulo `hybrid_search` manualmente
hybrid_search_path = os.path.join(SRC_PATH, "retrievers", "hybrid_search.py")
spec = importlib.util.spec_from_file_location("retrievers.hybrid_search", hybrid_search_path)
hybrid_search = importlib.util.module_from_spec(spec)
spec.loader.exec_module(hybrid_search)

# Asignar funciones y modelos a variables
embedding_model = reranking.embedding_model
generate_keywords_with_t5 = reranking.generate_keywords_with_t5
expand_keywords_with_similarity = reranking.expand_keywords_with_similarity
hybrid_reranking = hybrid_search.hybrid_reranking

print("Importaciones manuales de `retrievers` completadas.")

c:\Users\gigle\anaconda3\envs\mia\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Importaciones manuales de `retrievers` completadas.


In [4]:
# ------------------------------------------------------------
# Conectar a Qdrant
# ------------------------------------------------------------
from dotenv import load_dotenv
import os
from qdrant_client import QdrantClient

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Leer las variables desde .env
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
INDEX_NAME = "embeddings-version-2"

# Validar que las variables se cargaron correctamente
if not QDRANT_URL or not QDRANT_API_KEY:
    raise ValueError("Las variables QDRANT_URL o QDRANT_API_KEY no están configuradas correctamente en el archivo .env")

print("Conectando a Qdrant...")
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
print("Conexión a Qdrant establecida con éxito.")

Conectando a Qdrant...
Conexión a Qdrant establecida con éxito.


# Preprocessing

In [ ]:
# ------------------------------------------------------------
# Configurar las rutas de trabajo y mostrar detalles de los PDFs
# ------------------------------------------------------------
import pdfplumber

print("\n--- Configurando las rutas de trabajo ---")
CAPITULOS_DIR = os.path.join(BASE_DIR, "data/capitulos")
if not os.path.exists(CAPITULOS_DIR):
    raise FileNotFoundError(f"No se encontró el directorio 'data/capitulos' en {BASE_DIR}.")

# Obtener todos los PDFs en el directorio
pdf_paths = [
    os.path.join(CAPITULOS_DIR, file)
    for file in os.listdir(CAPITULOS_DIR)
    if file.endswith(".pdf")
]

# Mostrar resumen inicial de los archivos PDF y su cantidad de páginas
print(f"\n--- {len(pdf_paths)} PDFs encontrados en '{CAPITULOS_DIR}' ---")
for idx, pdf_path in enumerate(pdf_paths, start=1):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page_count = len(pdf.pages)
            print(f"{idx}. {os.path.basename(pdf_path)} - {page_count} páginas")
    except Exception as e:
        print(f"{idx}. {os.path.basename(pdf_path)} - Error al contar páginas: {e}")

In [ ]:
# ------------------------------------------------------------
# Procesar PDFs directamente
# ------------------------------------------------------------
print("\n--- Procesando TODOS los PDFs en el directorio ---")
results = process_pdf_directly(pdf_paths)

# Mostrar resultados del procesamiento
print("\n--- Resumen del procesamiento ---")
for pdf, data in results.items():
    print(f"{os.path.basename(pdf)}: {data['total_paginas']} páginas, {data['total_bloques']} bloques de texto, {data['total_tablas']} tablas.")

In [ ]:
# ------------------------------------------------------------
# Revisar tablas en documentos seleccionados y limitar cantidad
# ------------------------------------------------------------

# Lista de documentos a revisar y el número máximo de tablas por documento
documents_to_review = {
    "capitulo7.pdf": 3,  # Revisar hasta 3 tablas del documento capitulo7.pdf
    "capitulo14.pdf": 3,  # Revisar hasta 3 tablas del documento capitulo14.pdf
    "capitulo19.pdf": 3,  # Revisar hasta 3 tablas del documento capitulo19.pdf
    "capitulo22.pdf": 2,  # Revisar hasta 2 tablas del documento capitulo22.pdf
}

print("\n--- Revisando tablas en los documentos seleccionados ---")

# Iterar sobre cada documento y su límite de tablas
for pdf_name, max_tables in documents_to_review.items():
    # Construir la ruta completa del documento
    pdf_file = os.path.join(CAPITULOS_DIR, pdf_name)
    
    # Verificar si el archivo existe
    if not os.path.exists(pdf_file):
        print(f"Archivo no encontrado: {pdf_name}, saltando...")
        continue

    print(f"\nRevisando tablas en {pdf_name} (límite: {max_tables} tablas)")

    try:
        # Llamar a la función con el límite de tablas especificado
        tables = process_and_review_tables(pdf_file, max_tables=max_tables)
        
        # Mostrar el resultado del procesamiento
        print(f"Se procesaron {len(tables)} tablas en {pdf_name} (límite era {max_tables})")
    except Exception as e:
        # Capturar y mostrar cualquier error
        print(f"Error al procesar {pdf_name}: {e}")
    
    print("\n" + "="*80 + "\n")

In [ ]:
# ------------------------------------------------------------
# Inspeccionar contenido de los PDFs con un límite de caracteres
# ------------------------------------------------------------
print("\n--- Inspeccionando contenido de los PDFs con límite de caracteres ---")
inspect_pdf_content(CAPITULOS_DIR, pdf_paths, num_pages_limit=5, char_limit=500)

In [ ]:
from loaders.metadata_processor import process_pdfs_with_validated_metadata

# ------------------------------------------------------------
# Procesar los PDFs y generar datos enriquecidos
# ------------------------------------------------------------
print("\n--- Procesando PDFs y generando datos enriquecidos ---")
data_enriched = process_pdfs_with_validated_metadata(pdf_paths, verbose=False)

In [ ]:
# ------------------------------------------------------------
# Mostrar Resumen Global
# ------------------------------------------------------------
def mostrar_resumen_global(data_enriched, max_pdfs=5):
    """
    Muestra un resumen global del procesamiento de PDFs.

    Args:
        data_enriched (dict): Datos procesados enriquecidos.
        max_pdfs (int): Número máximo de PDFs a mostrar en el resumen.
    """
    print("\n--- Resumen Global del Procesamiento ---")
    for idx, (pdf_path, pdf_data) in enumerate(data_enriched.items(), start=1):
        total_paginas = len(pdf_data.get('paginas', []))
        total_capitulos = sum(
            1 for p in pdf_data['paginas'] for c in p['contenido']
            if c['tipo'] == 'texto' and 'CAPÍTULO' in c['texto']
        )
        total_tablas = sum(
            1 for p in pdf_data['paginas'] for c in p['contenido']
            if c['tipo'] == 'tabla'
        )

        print(f"\nArchivo {idx}: {os.path.basename(pdf_path)}")
        print(f"  Total de páginas procesadas: {total_paginas}")
        print(f"  Total de capítulos detectados: {total_capitulos}")
        print(f"  Total de tablas detectadas: {total_tablas}")

        if idx >= max_pdfs:
            print("... (se han procesado más archivos, pero se limitan los resultados mostrados)")
            break


# Llama a la función con un límite de 5 PDFs para mostrar
mostrar_resumen_global(data_enriched, max_pdfs=5)

In [ ]:
from loaders.metadata_verification import verificar_metadatos_guardados

# ------------------------------------------------------------
# Verificar metadatos guardados con límites
# ------------------------------------------------------------
print("\n--- Verificando Metadatos Guardados ---")
verificar_metadatos_guardados(data_enriched, max_files=3, max_pages=2, max_blocks=5)

In [ ]:
# ------------------------------------------------------------
# Importar módulos necesarios
# ------------------------------------------------------------
from loaders.review_tools import revisar_estructuras_generadas, revisar_paginas_especificas

# ------------------------------------------------------------
# Revisar ejemplos de estructuras generadas
# ------------------------------------------------------------
print("\n--- Revisando ejemplos de estructuras generadas ---")
revisar_estructuras_generadas(data_enriched, num_ejemplos=3)

In [ ]:
# ------------------------------------------------------------
# Revisar páginas específicas
# ------------------------------------------------------------
print("\n--- Revisando páginas específicas del archivo seleccionado ---")
revisar_paginas_especificas(data_enriched, "capitulo13", paginas_deseadas=list(range(6, 11)))

# Chunking

In [13]:
# ------------------------------------------------------------
# Importar módulos necesarios
# ------------------------------------------------------------
from chunking.separated_chunker import chunk_documents_separately
from chunking.chunk_verifier import verificar_chunks_narrativos_completos, verificar_chunks_tablas_completos
from chunking.chunk_analysis import analizar_chunks_narrativos, analizar_chunks_tablas, calcular_estadisticas_chunks_por_capitulo

In [ ]:
# ------------------------------------------------------------
# Validación de Datos Enriquecidos
# ------------------------------------------------------------
print("\n--- Verificando Datos Enriquecidos ---")
print(f"Total de archivos procesados: {len(data_enriched)}")
for archivo, contenido in data_enriched.items():
    print(f"Archivo: {archivo}, Total de páginas: {len(contenido['paginas'])}")

In [ ]:
# ------------------------------------------------------------
# Generar Chunks Separados
# ------------------------------------------------------------
print("\n--- Generando Chunks Narrativos y de Tablas ---")
chunks_narrativos, chunks_tablas = chunk_documents_separately(data_enriched, chunk_size=3000)

# Verificar resultados
print(f"Total de chunks narrativos: {len(chunks_narrativos)}")
print(f"Total de chunks de tablas: {len(chunks_tablas)}")

In [ ]:
# ------------------------------------------------------------
# Verificar Ejemplos de Chunks
# ------------------------------------------------------------
verificar_chunks_narrativos_completos(chunks_narrativos, num_chunks=3)
verificar_chunks_tablas_completos(chunks_tablas, num_chunks=3)

In [ ]:
# ------------------------------------------------------------
# Analizar Estadísticas Generales de los Chunks
# ------------------------------------------------------------
print("\n--- Analizando Estadísticas de los Chunks Narrativos ---")
estadisticas_narrativos = analizar_chunks_narrativos(chunks_narrativos)
print(f"  Total de chunks: {estadisticas_narrativos['total_chunks']}")
print(f"  Total de caracteres: {estadisticas_narrativos['total_caracteres']}")
print(f"  Promedio de caracteres por chunk: {estadisticas_narrativos['promedio_caracteres']:.2f}")

print("\n--- Analizando Estadísticas de los Chunks de Tablas ---")
estadisticas_tablas = analizar_chunks_tablas(chunks_tablas)
print(f"  Total de chunks de tablas: {estadisticas_tablas['total_chunks']}")
print(f"  Total de filas: {estadisticas_tablas['total_filas']}")
print(f"  Promedio de filas por tabla: {estadisticas_tablas['promedio_filas']:.2f}")
print(f"  Promedio de columnas por tabla: {estadisticas_tablas['promedio_columnas']:.2f}")

In [ ]:
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# Histograma para Chunks Narrativos (Tamaño en Caracteres)
# ------------------------------------------------------------
def generar_histograma_narrativos(chunks_narrativos):
    tamaños = [len(chunk["texto"]) for chunk in chunks_narrativos]

    plt.figure(figsize=(10, 6))
    plt.hist(tamaños, bins=30, edgecolor='black', alpha=0.7)
    plt.title("Distribución del Tamaño de Chunks Narrativos")
    plt.xlabel("Tamaño del Chunk (caracteres)")
    plt.ylabel("Cantidad de Chunks")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

# ------------------------------------------------------------
# Histograma para Chunks de Tablas (Filas y Columnas)
# ------------------------------------------------------------
def generar_histograma_tablas(chunks_tablas):
    filas = [chunk["metadatos"]["filas"] for chunk in chunks_tablas]
    columnas = [chunk["metadatos"]["columnas"] for chunk in chunks_tablas]

    # Histograma de Filas
    plt.figure(figsize=(10, 6))
    plt.hist(filas, bins=20, color='orange', edgecolor='black', alpha=0.7)
    plt.title("Distribución del Número de Filas en Tablas")
    plt.xlabel("Cantidad de Filas por Tabla")
    plt.ylabel("Cantidad de Tablas")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Histograma de Columnas
    plt.figure(figsize=(10, 6))
    plt.hist(columnas, bins=20, color='green', edgecolor='black', alpha=0.7)
    plt.title("Distribución del Número de Columnas en Tablas")
    plt.xlabel("Cantidad de Columnas por Tabla")
    plt.ylabel("Cantidad de Tablas")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

# ------------------------------------------------------------
# Generar Histogramas
# ------------------------------------------------------------
print("\n--- Generando Histogramas para Chunks Narrativos ---")
generar_histograma_narrativos(chunks_narrativos)

print("\n--- Generando Histogramas para Chunks de Tablas ---")
generar_histograma_tablas(chunks_tablas)

# Embeddings

In [ ]:
# Importar dependencias
from embedding.embedding_v2 import generate_embeddings_v2
from vector_store_client.vector_store_client_v2 import insert_embeddings_v2
from qdrant_client import QdrantClient
from tqdm import tqdm

In [ ]:
# ------------------------------------------------------------
# Preparar los Chunks para Generar Embeddings
# ------------------------------------------------------------
# Combinar todos los chunks narrativos y de tablas en un solo formato estándar
all_chunks = []

for chunk in chunks_narrativos:
    all_chunks.append({
        "type": "narrative",
        "content": chunk["texto"],
        "metadata": chunk["metadatos"]
    })

for chunk in chunks_tablas:
    tabla_contenido = "\n".join(["\t".join(map(str, row)) for row in chunk["tabla"]])
    all_chunks.append({
        "type": "table",
        "content": tabla_contenido,
        "metadata": chunk["metadatos"]
    })

print(f"Total de chunks preparados para embeddings: {len(all_chunks)}")

In [ ]:
# ------------------------------------------------------------
# Generar Embeddings y Metadatos
# ------------------------------------------------------------
from embedding.embedding_v2 import generate_embeddings_v2

all_embeddings = generate_embeddings_v2(all_chunks)
print(f"Total de embeddings generados: {len(all_embeddings)}")

In [ ]:
# Separar embeddings y metadatos en listas separadas
embeddings_list = []
metadata_list = []

# Generar embeddings y metadatos con validaciones
for idx, entry in enumerate(all_embeddings):
    try:
        # Validar estructura esperada
        if "vector" in entry and "default" in entry["vector"]:
            embedding = entry["vector"]["default"]
        else:
            raise KeyError(f"Claves faltantes en vector en índice {idx}: {entry}")

        if "payload" in entry:
            payload = entry["payload"]
        else:
            raise KeyError(f"Clave 'payload' faltante en índice {idx}: {entry}")

        # Validar longitud del embedding
        if len(embedding) != 384:
            raise ValueError(f"Embedding en índice {idx} tiene tamaño incorrecto: {len(embedding)}")

        # Agregar a las listas
        embeddings_list.append(embedding)
        metadata_list.append(payload)
    except KeyError as e:
        print(f"Error procesando embedding en el índice {idx}: {e}")
    except Exception as e:
        print(f"Error inesperado en el índice {idx}: {e}")

# Validar resultados
print(f"Total embeddings después del bucle: {len(embeddings_list)}")
print(f"Ejemplo de embedding: {embeddings_list[0] if embeddings_list else 'Ninguno'}")
print(f"Total metadatos después del bucle: {len(metadata_list)}")
print(f"Ejemplo de metadata: {metadata_list[0] if metadata_list else 'Ninguno'}")

In [ ]:
print(f"embeddings_list: {len(embeddings_list) if embeddings_list else 'None'}")
print(f"metadata_list: {len(metadata_list) if metadata_list else 'None'}")

In [24]:
if any(len(embedding) != 384 for embedding in embeddings_list):
    print("Error: Algunos embeddings no tienen la dimensión adecuada.")

if any(not isinstance(metadata, dict) for metadata in metadata_list):
    print("Error: Algunos metadatos no tienen el formato adecuado.")

In [ ]:
print("Ejemplo de embedding:", embeddings_list[0])
print("Ejemplo de metadato:", metadata_list[0])

In [ ]:
# Verificar detalles de la colección
collection_info = qdrant_client.get_collection(INDEX_NAME)
print(collection_info)

In [ ]:
for idx, (embedding, metadata) in enumerate(zip(embeddings_list, metadata_list)):
    point = {
        "id": idx,
        "vector": {"default": embedding},  # Clave 'default' requerida por Qdrant
        "payload": metadata
    }
    print(f"Punto {idx}: {point}")
    if idx == 0:  # Ver solo el primer punto para simplificar
        break

In [ ]:
test_point = {
    "id": 1,
    "vector": {"default": embeddings_list[0]},  # Embedding bajo el nombre 'default'
    "payload": metadata_list[0]
}

response = qdrant_client.upsert(
    collection_name="embeddings-version-2",  # Nombre correcto de la colección
    points=[test_point]
)

print("Respuesta de inserción manual:", response)

In [ ]:
print(type(embeddings_list[0]))
print(len(embeddings_list[0]))

In [ ]:
batch = [
    {
        "id": i,
        "vector": {"default": embeddings_list[i]},
        "payload": metadata_list[i]
    }
    for i in range(10)  # Prueba con los primeros 10 puntos
]

response = qdrant_client.upsert(
    collection_name="embeddings-version-2",
    points=batch
)
print("Respuesta de inserción de lote pequeño:", response)

In [ ]:
# Verificar datos antes de insertar
if embeddings_list and metadata_list:
    print(f"Ejemplo de embedding: {embeddings_list[0][:5]}... (Tamaño: {len(embeddings_list[0])})")
    print(f"Ejemplo de payload: {metadata_list[0]}")

    print(f"Insertando {len(embeddings_list)} embeddings en la colección 'embeddings-version-2' en lotes de 100...")
    insert_embeddings_v2(
        client=qdrant_client,
        index_name="embeddings-version-2",
        embeddings=embeddings_list,
        metadata_list=metadata_list,
        batch_size=100
    )
    print("Embeddings generados e insertados en Qdrant con éxito.")
else:
    print("No hay embeddings o metadatos para insertar.")

In [ ]:
# ------------------------------------------------------------
# Validar Resultados
# ------------------------------------------------------------
print("\n--- Validación de Embeddings ---")
if all_embeddings:
    num_valid_embeddings = len(embeddings_list)
    print(f"Embeddings válidos: {num_valid_embeddings}/{len(all_chunks)}")
    print("Ejemplo de embedding:")
    print(f"Tipo: {metadata_list[0]['type']}")
    print(f"Contenido: {metadata_list[0]['content'][:100]}...")
    print(f"Embedding: {embeddings_list[0][:5]}...")  # Mostrar primeros valores
else:
    print("No se generaron embeddings.")

# Retrieval

In [5]:
from embedding.embedding_v2 import generate_query_embedding,generate_query_embedding_openai
from vector_store_client.vector_store_search import search_embeddings, format_search_results
from vector_store_client.vector_store_client import search_qdrant

# ------------------------------------------------------------
# Definir la consulta y generar su embedding
# ------------------------------------------------------------

def embed_and_search(query, coleccion, limit=5, embedding="qdrant"):
    # Generar embedding para la consulta
    try:
        if embedding == "openai":
            query_embedding = generate_query_embedding_openai(query)
            if not isinstance(query_embedding, list) or len(query_embedding) != 1536:
                raise ValueError("El embedding de la consulta no tiene el formato esperado (lista con 1536 dimensiones).")
        elif embedding == "qdrant":
            query_embedding = generate_query_embedding(query)
            if not isinstance(query_embedding, list) or len(query_embedding) != 384:
                raise ValueError("El embedding de la consulta no tiene el formato esperado (lista con 384 dimensiones).")
        else:
            raise("Select embeeding openai or qdrant")

        # Validar el formato del embedding generado
        
    except Exception as e:
        print(f"Error al generar el embedding de la consulta: {e}")
        raise

    # ------------------------------------------------------------
    # Realizar la búsqueda en Qdrant
    # ------------------------------------------------------------
    index_name = coleccion  # Nombre de la colección en Qdrant

    try:
        results = search_qdrant(qdrant_client, index_name, query_embedding, limit=limit)
    except Exception as e:
        print(f"Error al realizar la búsqueda en Qdrant: {e}")
        results = []

    return results
    
    
query = "¿Qué se entiende por cerveza?"

results = embed_and_search(query,"openai-750-overlap",limit=5, embedding="openai")

Modelo 'all-MiniLM-L6-v2' cargado correctamente.
Cargando el módulo 'embedding'
Modelo 'all-MiniLM-L6-v2' cargado correctamente.
Generando embeddings con OpenAI para 1 chunks...


Generando embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.13chunk/s]


Total de embeddings generados con OpenAI: 1


In [6]:
chunks_basic = [point.payload['content'] for point in results]
print("\n---\n".join(chunks_basic))

CAPÍTULO XIII
BEBIDAS FERMENTADAS
CERVEZAS
Artículo 1080 - (Resolución Conjunta SPRyRS N° 63/02 y SAGPyA N° 345/02)
1. DESCRIPCIÓN
1.1 DEFINICIONES
1.1.1 Cerveza
Se entiende exclusivamente por cerveza la bebida resultante de fermentar, mediante
levadura cervecera, al mosto de cebada malteada o de extracto de malta, sometido
previamente a un proceso de cocción, adicionado de lúpulo. Una parte de la cebada
malteada o de extracto de malta podrá ser reemplazada por adjuntos cerveceros.
La cerveza negra podrá ser azucarada.
La cerveza podrá ser adicionada de colorantes, saborizantes y aromatizantes.
1.1.2 Malta Líquida
Se entiende por malta líquida la bebida no alcohólica, resultante del mosto de cebada
---
1.2.4.1 Cerveza
Es la cerveza elaborada a partir de un mosto cuyo extracto primitivo contiene un
mínimo de 55% en peso de cebada malteada.
1.2.4.2 Cerveza 100% malta o de pura malta
Es la cerveza elaborada a partir de un mosto cuyo extracto primitivo proviene
exclusivamente de cebada mal

In [7]:
print(results[0].payload['content'])

CAPÍTULO XIII
BEBIDAS FERMENTADAS
CERVEZAS
Artículo 1080 - (Resolución Conjunta SPRyRS N° 63/02 y SAGPyA N° 345/02)
1. DESCRIPCIÓN
1.1 DEFINICIONES
1.1.1 Cerveza
Se entiende exclusivamente por cerveza la bebida resultante de fermentar, mediante
levadura cervecera, al mosto de cebada malteada o de extracto de malta, sometido
previamente a un proceso de cocción, adicionado de lúpulo. Una parte de la cebada
malteada o de extracto de malta podrá ser reemplazada por adjuntos cerveceros.
La cerveza negra podrá ser azucarada.
La cerveza podrá ser adicionada de colorantes, saborizantes y aromatizantes.
1.1.2 Malta Líquida
Se entiende por malta líquida la bebida no alcohólica, resultante del mosto de cebada


In [9]:
eval_df = pd.read_excel('evaluacion.xlsx')
eval_df

,query,ground_truth,capitulo
0,¿Qué se entiende por cerveza?,Se entiende exclusivamente por cerveza la bebi...,13
1,¿Cómo se clasifican las cervezas?,El contenido de nutrientes y/o del valor energ...,13
2,¿Qué se define como sidra?,SIDRAS\nArtículo 1085 - (Resolución Conjunta S...,13
3,¿Se pueden añadir aromas a los vinos?,"Artículo 1105\nQueda prohibido fabricar, expon...",13
4,¿Los condimentos vegetales tienen algunas rest...,CONDIMENTOS VEGETALES\nArtículo 1199\nCon la d...,16
5,¿Qué caracteristicas debe tener la harina de c...,"“Con la denominación de Harina de Chía, se ent...",19
6,¿Quiénes deben cumplir con la regulación alime...,"Artículo 1\nToda persona, firma comercial o es...",1
7,¿Qué se define como aditivo alimentario?,Aditivo alime ntario: es cualquier ingrediente...,1
8,¿Cuáles son los límites de metales aceptados e...,"Artículo 156 - (Res. 1546, 17.9.85)\n""En los a...",3
9,¿Qué antioxidantes o sinergistas pueden agrega...,"Artículo 523bis - (Res 2012, 19.10.84)\n""Los a...",7


In [10]:
# eval_df_ = eval_df[:2]
# eval_df_

In [11]:
import answering.answering as ans
import retrievers.summarization as summ
import evaluation.evaluation_ragas as eval

coleccion = "openai-750-overlap"
embedding="openai"
limit = 30

# Initialize an empty list to collect results for each iteration
eval_results = []

for _, row in eval_df.iterrows():
    query = row.iloc[0]  # Assuming the query is in the first column
    ground_truth = row.iloc[1]
    print(f'Query {_}/{len(eval_df)}: {query}')
    
    # Perform embedding and search
    results = embed_and_search(query, coleccion, limit=limit, embedding=embedding)
    chunks_basic = [point.payload['content'] for point in results]
    chunks_adv = summ.summarize_multiple_chunks(query, chunks_basic)
    chunks_basic = chunks_basic[:min(len(chunks_adv),15)]
    
    # Generate answers
    answer_basic = ans.answer_query_with_context(query, chunks_basic)
    answer_adv = ans.answer_query_with_context(query, chunks_adv)
    
    # Evaluate RAGAS for basic and advanced retrieval
    ragas_basic = eval.evaluate_ragas(query, answer_basic, chunks_basic, ground_truth)
    ragas_basic['retrieval'] = "Basic"
    
    ragas_adv = eval.evaluate_ragas(query, answer_adv, chunks_adv, ground_truth)
    ragas_adv['retrieval'] = "Advanced"
    
    # Append results to the list
    eval_results.append(ragas_basic)
    eval_results.append(ragas_adv)

# Consolidate all results into a single DataFrame
df_eval = pd.concat(eval_results, ignore_index=True)

# Display or return the final DataFrame
print(df_eval)
    


Query 0/10: ¿Qué se entiende por cerveza?
Generando embeddings con OpenAI para 1 chunks...


Generando embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.43chunk/s]


Total de embeddings generados con OpenAI: 1
Query: ¿Qué se entiende por cerveza?


Processing chunks:   3%|▎         | 1/30 [00:02<01:12,  2.51s/it]

Relevant Summary | Se entiende exclusivamente por cerveza la bebida resultante de fermentar, mediante levadura cervecera, al mosto de cebada malteada o de extracto de malta, sometido previamente a un proceso de cocción, adicionado de lúpulo. Una parte de la cebada malteada o de extracto de malta podrá ser reemplazada por adjuntos cerveceros. La cerveza negra podrá ser azucarada. La cerveza podrá ser adicionada de colorantes, saborizantes y aromatizantes.


...

Relevant Summary | Para los aceites y grasas vegetales comestibles, se pueden agregar los siguientes antioxidantes:

- Palmitato de ascorbilo (INS 304)
- Estearato de ascorbilo (INS 305)
- Galato de propilo (INS 310)
- Resina de guayaco (INS 314)
- Ter butil hidroxiquinona, TBHQ (INS 319)
- Butil hidroxianisol, BHA (INS 320)
- Butil hidroxitolueno, BHT (INS 321)
- Ácido fosfórico (INS 338)
- Citrato de isopropilo (mezcla) (INS 384)


Processing chunks: 100%|██████████| 30/30 [00:27<00:00,  1.10it/s]


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

                                           user_input  \
0                       ¿Qué se entiende por cerveza?   
1                       ¿Qué se entiende por cerveza?   
2                   ¿Cómo se clasifican las cervezas?   
3                   ¿Cómo se clasifican las cervezas?   
4                          ¿Qué se define como sidra?   
5                          ¿Qué se define como sidra?   
6               ¿Se pueden añadir aromas a los vinos?   
7               ¿Se pueden añadir aromas a los vinos?   
8   ¿Los condimentos vegetales tienen algunas rest...   
9   ¿Los condimentos vegetales tienen algunas rest...   
10  ¿Qué caracteristicas debe tener la harina de c...   
11  ¿Qué caracteristicas debe tener la harina de c...   
12  ¿Quiénes deben cumplir con la regulación alime...   
13  ¿Quiénes deben cumplir con la regulación alime...   
14           ¿Qué se define como aditivo alimentario?   
15           ¿Qué se define como aditivo alimentario?   
16  ¿Cuáles son los límites de 

In [12]:
df_eval.to_csv(f'evaluacion_{coleccion}_{embedding}_4o.csv', index=False)

In [53]:
from datasets import Dataset 
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, LLMContextRecall
import os

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

def evaluate_ragas(query, answer,chunks, ground_truth):
    data = {
        'question': [
            query
        ],
        'response': [
                answer
        ],
        'contexts': [
                chunks
        ],
        'ground_truth': [
            ground_truth
        ]
    }

    dataset = Dataset.from_dict(data)

    context_recall = LLMContextRecall()
    score = evaluate(dataset, metrics=[context_precision, faithfulness, answer_relevancy, context_recall])
    df = score.to_pandas()
    return df

In [ ]:
ground_truth = '''Se considera polen no apto para el consumo, aquel que presente una o más de las siguientes
    características:
    1. Caracteres organolépticos anormales
    2. Exceso de polvillo o de propóleos
    3. Anormalidades en la observación microscópica
    4. Composición analítica diferente a la consignada anteriormente
    5. Características microbiológicas superiores a los límites establecidos
    6. Ataque de insectos, parásitos o sus larvas
    7. Residuos de plaguicidas
    8. Substancias conservadoras
    9. Impurezas no retenidas por un tamiz IRAM 500 μ (N° 35) más de 5 por 1000.'''


df_ragas = evaluate_ragas(query,answer_adv, chunks_adv, ground_truth)
df_ragas

In [ ]:
df_ragas = evaluate_ragas(query,answer_basic, chunks_basic, ground_truth)
df_ragas

In [ ]:
answer_adv = answer_query_with_context(query,chunks_adv)
answer_adv

In [ ]:
answer_basic = answer_query_with_context(query,chunks_basic)
answer_basic